In [1]:
# Install below packages if needed
!pip install nltk
!pip install cmudict

In [44]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from lexicalrichness import LexicalRichness
from nltk.stem import WordNetLemmatizer

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import warnings
from string import punctuation

from nltk.tokenize.api import TokenizerI
from nltk.util import ngrams

import cmudict
from collections import Counter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
asian = pd.read_csv("/DATA/Processed/asian.csv")
black = pd.read_csv("/DATA/Processed/blackPeople.csv")
white = pd.read_csv("/DATA/Processed/whitePeople.csv")
native = pd.read_csv("/DATA/Processed/nativeAmerican.csv")

In [10]:
print(black['body'][50:70])

50    At least two of his kids don’t have to worry a...
51    No. But I had friends I a ton of time with and...
52    Rule #1. This is a sub for the black community...
53                                                 Yes.
54    I voluntarily signed for myself out of high-sc...
55    All of this. And hugs were never really a thin...
56    Nope. Not as a kid\n\n\nLater on when I was an...
57    I’m starting to think that the majority of ppl...
58    So basically the EWF an American organization ...
59    Nick Canon is trying to promote animal husband...
60                                   Great interview...
61    So firstly, there is a distinction between con...
62                             This! ALL of this! THIS!
63    why? more Native Americans died, and we are no...
64    Memphis is the 26th most dangerous city in the...
65        I could absolutely do that! I’ll message you!
66    &gt;but they were allowed to at least exist un...
67    I figured... I do not live anywhere near M

In [11]:
#data = pd.read_csv("/content/drive/MyDrive/DATA/Race/Processed/race.csv")

In [12]:
#print(data.columns)
#print(len(data))

In [13]:
def get_total_length(df, x):
  len = df[x].str.len().sum()
  return len

In [14]:
def get_mean(df):
  return df['body'].str.len().mean()

def get_std(df):
  return df['body'].str.len().std()

In [15]:
len_asian = get_total_length(asian, 'body')
len_black = get_total_length(black, 'body')
len_white = get_total_length(white, 'body')
len_native = get_total_length(native, 'body')

In [16]:
print(len_asian)
print(len_black)
print(len_white)
print(len_native)

23774794
41295450
15663327
20589033


Then we calculate the average length of the text for each class.

In [17]:
asian_mean = get_mean(asian)
black_mean = get_mean(black)
white_mean = get_mean(white)
native_mean = get_mean(native)
print(white_mean)
print(black_mean)
print(asian_mean)
print(native_mean)

159.034693877551
438.9957264957265
254.75541125541125
211.63842975206612


In [18]:
asian_std = get_std(asian)
black_std = get_std(black)
white_std = get_std(white)
native_std = get_std(native)
print(white_std)
print(black_std)
print(asian_std)
print(native_std)

In [19]:
asian['text_len_norm'] = (asian['body'].str.len()-asian_mean)/asian_std
black['text_len_norm'] = (black['body'].str.len()-black_mean)/black_std
white['text_len_norm'] = (white['body'].str.len()-white_mean)/white_std
native['text_len_norm'] = (native['body'].str.len()-native_mean)/native_std

In [20]:
print(black['body'][3])

You’re doing what’s best for you and that’s a good thing.  If you feel you don’t need it at this time, then yes, cut it off.  I think your mom is trying to “prepare you for the future” (lol) in her own way but need to understand that you’re an adult and you have to do what you need to take care of yourself.


In [21]:
temp = asian.append(black)
temp1 = temp.append(white)
data = temp1.append(native)

data.reset_index(inplace=True, drop=True)

<ipython-input-21-d2e06a854ba8>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = asian.append(black)
<ipython-input-21-d2e06a854ba8>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp1 = temp.append(white)
<ipython-input-21-d2e06a854ba8>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = temp1.append(native)


In [22]:
data['text_length'] = data['body'].str.len()

In [23]:
# Average word embedding vector score
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action = 'ignore')
 
import gensim
from gensim.models import Word2Vec
'''from gensim.parsing.preprocessing import STOPWORDS

from gensim import corpora
from gensim.utils import simple_preprocess'''

from nltk.corpus import stopwords

In [24]:
stop_words = set(stopwords.words('english'))

In [25]:
print(stop_words)

{'wasn', 'on', 'not', 'were', 'we', 'she', 'if', 'because', 'that', 'some', 're', "doesn't", 'i', 'itself', 'between', 'herself', 'as', 'off', 'more', 'with', "should've", 'whom', 'couldn', 'you', 'will', 'our', 'such', 'your', 'here', 'isn', 'shouldn', 'an', 'why', "didn't", 'by', 'against', 'are', 'who', 'his', 'does', "mustn't", "it's", 'these', 'and', 'from', 'd', "wasn't", "you're", 'own', 'which', 'or', 'once', 'mightn', 'those', 'weren', 'her', 'theirs', 'further', 'at', 'in', 'have', "shouldn't", 'all', 'just', "haven't", "isn't", 'each', 'aren', 'them', 'wouldn', 'ain', 'below', 'both', 'has', 'during', "mightn't", "won't", 'myself', 'now', 'under', 'don', 'himself', 'm', "you'd", 'y', 'same', 'mustn', 'him', 'only', 'most', 'themselves', 'when', 'it', 'o', 'they', 'about', 'hadn', 'll', "weren't", 'before', 'again', 've', "needn't", 'hers', 'ma', 'did', 'doesn', 'having', 'other', 'very', "you'll", 'so', 'how', 's', 'through', 'then', 'nor', 'can', 'won', "wouldn't", 'the', '

In [26]:
def process_text(data):
    
    text = []
    for i in tqdm(range(len(data))):
      if data[i] !="":
        # Sentence tokenization using NLTK library
          temp = []

          for word in word_tokenize(data[i]):
             if word not in stop_words:
              temp.append(word.lower())
                

      text.append(temp)


      '''for i in sent_tokenize(text):
        temp = []
        # tokenize the sentence into words
        for j in word_tokenize(i):
            if j not in STOPWORDS:
              temp.append(j.lower())
              #temp.append(j.lower())
              data.append(temp)'''
    
    return text

In [27]:
texts = data['cleaned_body'].values

In [28]:
data_words = process_text(texts)

100%|██████████| 383166/383166 [01:53<00:00, 3376.27it/s]


In [29]:
print(data['cleaned_body'][10])
#print(data_words)

Sothen just do what he suggested and remove the problematic comments and leave the rest of the thread  Im perplexed as to how thats a problem


In [30]:
print(len(data_words))

383166


In [31]:
data['token_text'] = data_words

In [32]:
print(data['token_text'])

0         [gt, but, retirement, privilege, often, one, b...
1         [singaporean, authorities, wont, bother, youre...
2                                                    [whos]
3         [ive, called, paranoid, bringing, too, many, c...
4                     [i, live, west, we, definitely, post]
                                ...                        
383161    [i, think, clearly, missing, picture, magnific...
383162    [ignorant, european, did, enter, see, owners, ...
383163    [as, white, metal, head, hillbilly, guy, growi...
383164    [consultation, inquiries, frequently, asked, a...
383165    [only, $, 15k, she, deserves, much, like, leas...
Name: token_text, Length: 383166, dtype: object


In [33]:
model = gensim.models.Word2Vec(data_words, min_count = 1,vector_size=100, window = 5)

In [34]:
# Count vectorization of text
'''from sklearn.feature_extraction.text import CountVectorizer
  
# Creating the vectorizer
vectorizer = CountVectorizer(stop_words='english')
 
# Converting the text to numeric data
X = vectorizer.fit_transform(texts)
 
#print(vectorizer.get_feature_names())
 
# Preparing Data frame For machine learning
# Priority column acts as a target variable and other columns as predictors
CountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
CountVectorizedData['Race']=data['race']
print(CountVectorizedData.shape)
CountVectorizedData.head()'''

"from sklearn.feature_extraction.text import CountVectorizer\n  \n# Creating the vectorizer\nvectorizer = CountVectorizer(stop_words='english')\n \n# Converting the text to numeric data\nX = vectorizer.fit_transform(texts)\n \n#print(vectorizer.get_feature_names())\n \n# Preparing Data frame For machine learning\n# Priority column acts as a target variable and other columns as predictors\nCountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())\nCountVectorizedData['Race']=data['race']\nprint(CountVectorizedData.shape)\nCountVectorizedData.head()"

In [35]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.vectors[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [36]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(model)
mean_embedded = mean_embedding_vectorizer.fit_transform(data['token_text'])

In [37]:
x = (np.mean(mean_embedded,axis=1))
data['mean_w2v'] = x

In [38]:
print(data['mean_w2v'].describe())

count    383166.000000
mean          0.041678
std           0.105415
min          -0.919145
25%          -0.010875
50%           0.046103
75%           0.099156
max           0.529365
Name: mean_w2v, dtype: float64


In [39]:
print(data.columns)

Index(['subreddit_id', 'author', 'body', 'cleaned_body', 'race',
       'text_len_norm', 'text_length', 'token_text', 'mean_w2v'],
      dtype='object')


In [40]:
#data.to_csv("/content/drive/MyDrive/DATA/Race/Processed/race_data.csv",index=False)
#data.to_csv("/content/drive/MyDrive/DATA/Race/Processed/New/race_data.csv",index=False)

In [41]:
#!pip install lexical-diversity
!pip install lexicalrichness

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lexicalrichness: filename=lexicalrichness-0.5.0-py3-none-any.whl size=15534 sha256=c6c69d3c24aaef359850c664c2d27f2342cc6ecffb86abb5bddbecece847d292
  Stored in directory: /root/.cache/pip/wheels/42/68/59/1edd70c2b91dc172fa208eb34799e90bc6c093bfbb862ff017
Successfully built lexicalrichness


In [43]:
#data = pd.read_csv("/content/drive/MyDrive/DATA/Race/Processed/race_data.csv")
data = pd.read_csv("/DATA/Processed/race_data.csv")

In [47]:
# import these modules

  
lemmatizer = WordNetLemmatizer()

In [48]:
'''l = []
for i in tqdm(range(0,len(data)),desc = 'Progress'):
  #print(x)
  lemm_text = lemmatizer.lemmatize(data['cleaned_body'][i])
  l.append(ld.mtld(lemm_text))'''

"l = []\nfor i in tqdm(range(0,len(data)),desc = 'Progress'):\n  #print(x)\n  lemm_text = lemmatizer.lemmatize(data['cleaned_body'][i])\n  l.append(ld.mtld(lemm_text))"

In [49]:
'''def lemm_process(text):
    lemm_text = lemmatizer.lemmatize(text)
    return lemm_text
data['lemm_tokens'] = data['token_text'].apply(lemm_process)'''

"def lemm_process(text):\n    lemm_text = lemmatizer.lemmatize(text)\n    return lemm_text\ndata['lemm_tokens'] = data['token_text'].apply(lemm_process)"

In [50]:
data['cleaned_body'].isnull().any()

False

In [51]:
print(data['token_text'])

0         ['yeah', 'honestly', 'even', 'black', 'person'...
1         ['the', 'opposite', 'experience', 'rarely', 'f...
2         ['memphis', '26th', 'dangerous', 'city', 'enti...
3         ['compare', 'message', 'wandering', 'earth', '...
4         ['if', 'youre', 'serious', 'semiserious', 'cou...
                                ...                        
369489                            ['thanks', 'ill', 'tell']
369490    ['i', 'heard', 'black', 'guy', 'canada', 'tell...
369491    ['did', 'report', 'youtube', 'administration',...
369492                                  ['zebra', 'people']
369493                        ['i', 'get', 'hell', 'state']
Name: token_text, Length: 369494, dtype: object


In [52]:
data = data[~data['cleaned_body'].str.isnumeric()]
data.reset_index(inplace=True, drop=True)

In [53]:
print(data['cleaned_body'])

0         Yeah honestly even as a black person from up n...
1         The opposite experience here have rarely found...
2         Memphis is the 26th most dangerous city in the...
3         compare the message from wandering earth 2 and...
4         If youre serious or semiserious you could look...
                                ...                        
369489                                  Thanks Ill tell her
369490    I heard a black guy here in Canada telling me ...
369491    Did you report it to YouTube administration Th...
369492                                         Zebra people
369493             I have to get the hell out of this state
Name: cleaned_body, Length: 369494, dtype: object


In [54]:

tqdm.pandas()

def mtld(text):
    lex = LexicalRichness(text)
    return lex.mtld()

data['ld_mtld'] = data['cleaned_body'].progress_apply(mtld)


100%|██████████| 369494/369494 [00:42<00:00, 8780.00it/s] 


In [55]:
print(data['ld_mtld'])

0          95.873689
1         134.560000
2          73.173333
3          11.000000
4         116.640000
             ...    
369489      4.000000
369490    224.000000
369491     13.000000
369492      2.000000
369493     10.000000
Name: ld_mtld, Length: 369494, dtype: float64


In [56]:
def maas(text):
    try:
      lex = LexicalRichness(text)
      output = lex.Maas
    except ZeroDivisionError:
      output = "" 
    return output

data['ld_maas'] = data['cleaned_body'].progress_apply(maas)

100%|██████████| 369494/369494 [00:11<00:00, 30832.86it/s]


In [57]:
def hdd(text):
    lex = LexicalRichness(text)
    return lex.hdd(draws=1)

data['ld_hdd'] = data['cleaned_body'].progress_apply(hdd)

100%|██████████| 369494/369494 [45:56<00:00, 134.03it/s]


In [58]:
nltk.downloader.download('vader_lexicon')

senti = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [59]:
text_sent = []
for i in tqdm(range(0,len(data)),desc = 'Progress'):
  text_sent.append(senti.polarity_scores(data['cleaned_body'][i]))

Progress: 100%|██████████| 369494/369494 [04:04<00:00, 1511.09it/s]


In [60]:
print(text_sent[:5])

[{'neg': 0.179, 'neu': 0.713, 'pos': 0.108, 'compound': -0.9966}, {'neg': 0.153, 'neu': 0.806, 'pos': 0.04, 'compound': -0.8074}, {'neg': 0.057, 'neu': 0.902, 'pos': 0.042, 'compound': -0.2247}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.011, 'neu': 0.842, 'pos': 0.147, 'compound': 0.9393}]


In [61]:
data['sentiment_scores'] = text_sent

In [62]:
#data.to_csv("/content/drive/MyDrive/DATA/Race/Processed/New/race_data.csv",index=False)

In [63]:
print(data.columns)

Index(['subreddit_id', 'author', 'body', 'cleaned_body', 'race',
       'text_len_norm', 'text_length', 'token_text', 'mean_w2v', 'ld_mtld',
       'ld_maas', 'ld_hdd', 'syllable_count', 'percent_diff_words', 'NN', 'CC',
       'VBZ', 'DT', 'RB', 'CD', 'WDT', 'JJ', 'NNS', 'IN', 'PRP$', 'NNP', 'VBN',
       'TO', 'VBG', 'WP', 'WRB', 'VBD', 'NNPS', 'VBP', 'JJS', 'PRP', 'VB',
       'JJR', 'MD', 'RP', 'RBR', 'EX', 'FW', 'RBS', 'PDT', 'UH', '$', 'WP$',
       'LS', 'pos_sent', 'neg_sent', 'neu_sent', 'sentiment_scores'],
      dtype='object')


In [65]:
#!zip -r /content/drive/MyDrive/DATA/Race/Processed/New/race_data.zip /content/drive/MyDrive/DATA/Race/Processed/New/race_data.csv

In [66]:
#data = pd.read_csv("/content/drive/MyDrive/DATA/Race/Processed/New/race_data.csv")

In [67]:
# Natural Language Toolkit: Tokenizers
#
# Copyright (C) 2001-2020 NLTK Project
# Author: Christopher Hench <chris.l.hench@gmail.com>
#         Alex Estes
# URL: <http://nltk.sourceforge.net>
# For license information, see LICENSE.TXT

"""
The Sonority Sequencing Principle (SSP) is a language agnostic algorithm proposed
by Otto Jesperson in 1904. The sonorous quality of a phoneme is judged by the
openness of the lips. Syllable breaks occur before troughs in sonority. For more
on the SSP see Selkirk (1984).

The default implementation uses the English alphabet, but the `sonority_hiearchy`
can be modified to IPA or any other alphabet for the use-case. The SSP is a
universal syllabification algorithm, but that does not mean it performs equally
across languages. Bartlett et al. (2009) is a good benchmark for English accuracy
if utilizing IPA (pg. 311).

Importantly, if a custom hiearchy is supplied and vowels span across more than
one level, they should be given separately to the `vowels` class attribute.

References:
- Otto Jespersen. 1904. Lehrbuch der Phonetik.
  Leipzig, Teubner. Chapter 13, Silbe, pp. 185-203.
- Elisabeth Selkirk. 1984. On the major class features and syllable theory.
  In Aronoff & Oehrle (eds.) Language Sound Structure: Studies in Phonology.
  Cambridge, MIT Press. pp. 107-136.
- Susan Bartlett, et al. 2009. On the Syllabification of Phonemes.
  In HLT-NAACL. pp. 308-316.
"""


class SyllableTokenizer(TokenizerI):
    """
    Syllabifies words based on the Sonority Sequencing Principle (SSP).

        >>> from nltk.tokenize import SyllableTokenizer
        >>> from nltk import word_tokenize
        >>> SSP = SyllableTokenizer()
        >>> SSP.tokenize('justification')
        ['jus', 'ti', 'fi', 'ca', 'tion']
        >>> text = "This is a foobar-like sentence."
        >>> [SSP.tokenize(token) for token in word_tokenize(text)]
        [['This'], ['is'], ['a'], ['foo', 'bar', '-', 'li', 'ke'], ['sen', 'ten', 'ce'], ['.']]
    """

    def __init__(self, lang="en", sonority_hierarchy=False):
        """
        :param lang: Language parameter, default is English, 'en'
        :type lang: str
        :param sonority_hierarchy: Sonority hierarchy according to the
                                   Sonority Sequencing Principle.
        :type sonority_hierarchy: list(str)
        """
        # Sonority hierarchy should be provided in descending order.
        # If vowels are spread across multiple levels, they should be
        # passed assigned self.vowels var together, otherwise should be
        # placed in first index of hierarchy.
        if not sonority_hierarchy and lang == "en":
            sonority_hierarchy = [
                "aeiouy",  # vowels.
                "lmnrw",  # nasals.
                "zvsf",  # fricatives.
                "bcdgtkpqxhj",  # stops.
            ]

        self.vowels = sonority_hierarchy[0]
        self.phoneme_map = {}
        for i, level in enumerate(sonority_hierarchy):
            for c in level:
                sonority_level = len(sonority_hierarchy) - i
                self.phoneme_map[c] = sonority_level
                self.phoneme_map[c.upper()] = sonority_level

    def assign_values(self, token):
        """
        Assigns each phoneme its value from the sonority hierarchy.
        Note: Sentence/text has to be tokenized first.

        :param token: Single word or token
        :type token: str
        :return: List of tuples, first element is character/phoneme and
                 second is the soronity value.
        :rtype: list(tuple(str, int))
        """
        syllables_values = []
        for c in token:
            try:
                syllables_values.append((c, self.phoneme_map[c]))
            except KeyError:
                if c not in punctuation:
                    warnings.warn(
                        "Character not defined in sonority_hierarchy,"
                        " assigning as vowel: '{}'".format(c)
                    )
                    syllables_values.append((c, max(self.phoneme_map.values())))
                    self.vowels += c
                else:  # If it's a punctuation, assing -1.
                    syllables_values.append((c, -1))
        return syllables_values


    def validate_syllables(self, syllable_list):
        """
        Ensures each syllable has at least one vowel.
        If the following syllable doesn't have vowel, add it to the current one.

        :param syllable_list: Single word or token broken up into syllables.
        :type syllable_list: list(str)
        :return: Single word or token broken up into syllables
                 (with added syllables if necessary)
        :rtype: list(str)
        """
        valid_syllables = []
        front = ""
        for i, syllable in enumerate(syllable_list):
            if syllable in punctuation:
                valid_syllables.append(syllable)
                continue
            if not re.search("|".join(self.vowels), syllable):
                if len(valid_syllables) == 0:
                    front += syllable
                else:
                    valid_syllables = valid_syllables[:-1] + [
                        valid_syllables[-1] + syllable
                    ]
            else:
                if len(valid_syllables) == 0:
                    valid_syllables.append(front + syllable)
                else:
                    valid_syllables.append(syllable)

        return valid_syllables


    def tokenize(self, token):
        """
        Apply the SSP to return a list of syllables.
        Note: Sentence/text has to be tokenized first.

        :param token: Single word or token
        :type token: str
        :return syllable_list: Single word or token broken up into syllables.
        :rtype: list(str)
        """
        # assign values from hierarchy
        syllables_values = self.assign_values(token)

        # if only one vowel return word
        if sum(token.count(x) for x in self.vowels) <= 1:
            return [token]

        syllable_list = []
        syllable = syllables_values[0][0]  # start syllable with first phoneme
        for trigram in ngrams(syllables_values, n=3):
            phonemes, values = zip(*trigram)
            # Sonority of previous, focal and following phoneme
            prev_value, focal_value, next_value = values
            # Focal phoneme.
            focal_phoneme = phonemes[1]

            # These cases trigger syllable break.
            if focal_value == -1:  # If it's a punctuation, just break.
                syllable_list.append(syllable)
                syllable_list.append(focal_phoneme)
                syllable = ""
            elif prev_value >= focal_value == next_value:
                syllable += focal_phoneme
                syllable_list.append(syllable)
                syllable = ""

            elif prev_value > focal_value < next_value:
                syllable_list.append(syllable)
                syllable = ""
                syllable += focal_phoneme

            # no syllable break
            else:
                syllable += focal_phoneme

        syllable += syllables_values[-1][0]  # append last phoneme
        syllable_list.append(syllable)

        return self.validate_syllables(syllable_list)

In [69]:

d = cmudict.dict()
syl = []

def nsyl(word):
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    except:
        st = SyllableTokenizer()
        return len(st.tokenize(word))

def syl_count(i):
  tokens = data['token_text'][i]
  syl_tokens = [nsyl(t) for t in tokens]
  syl_count_token = sum(syl_tokens)
  return sum(syl_tokens)

for i in tqdm(range(len(data['cleaned_body'])), desc='Progress'):
    syl.append(syl_count(i))

Progress: 100%|██████████| 369494/369494 [12:02<00:00, 511.69it/s]


In [70]:
data['syllable_count'] = syl

In [ ]:
diff_words = []
def tokens_no_nums(txt):
  txt = re.sub('\d', '', txt)
  tokens = nltk.word_tokenize(txt)
  words = [word for word in tokens if word.isalpha()]
  return words

def token_count(txt):
  return len(tokens_no_nums(txt))

def more_2_syl(txt):
  count = 0
  tokens = tokens_no_nums(txt)
  syl_tokens = [nsyl(t) for t in tokens]
  for s in syl_tokens:
    if s > 2:
      count += 1
  return count

def per_more_2_syl(txt):
  if token_count(txt) != 0:
    return 100*more_2_syl(txt)/token_count(txt)
  else:
    return ""

for i in tqdm(range(0, len(data)), desc='Progress'):
  
  diff_words.append(per_more_2_syl(data['cleaned_body'][i]))

Progress:   3%|▎         | 10278/369494 [00:21<20:50, 287.19it/s]

In [ ]:
print(token_count(data['body'][187500]))

In [ ]:
print(data['body'][353])

In [ ]:
print(data['body'][187603])
print(data['token_text'][187603])

In [ ]:
data['percent_diff_words'] = diff_words

In [ ]:
print(data['token_text'][0])
y = data['token_text'][0]
x = nltk.pos_tag(['gt', 'but', 'retirement', 'privilege', 'often'])
print(x)

In [78]:

tqdm.pandas()

#data['token'] = data.apply(lambda row: nltk.word_tokenize(row['STORY']), axis=1)
data['pos_tags'] = data.progress_apply(lambda row: nltk.pos_tag(nltk.word_tokenize(row['cleaned_body'])), axis=1)


tag_count_data = pd.DataFrame(data['pos_tags'].map(lambda x: Counter(tag[1] for tag in x)).to_list())

pd.concat([data, tag_count_data], axis=1).fillna(0).drop(['pos_tags'], axis=1)

  0%|          | 0/369494 [00:00<?, ?it/s]

,subreddit_id,author,body,cleaned_body,race,text_len_norm,text_length,token_text,mean_w2v,ld_mtld,...,WDT,FW,JJR,PDT,NNPS,RBR,RBS,$,WP$,LS
0,t5_2qr1c,Agreeable_Task500,"Yeah honestly, even as a black person from up ...",Yeah honestly even as a black person from up n...,Native American,9.547114,4552,"['yeah', 'honestly', 'even', 'black', 'person'...",0.057087,95.873689,...,5.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,t5_2rfyw,thefumingo,The opposite experience here; have rarely foun...,The opposite experience here have rarely found...,Asian,0.243500,332,"['the', 'opposite', 'experience', 'rarely', 'f...",-0.008365,134.560000,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,t5_2r81q,joculator,Memphis is the 26th most dangerous city in the...,Memphis is the 26th most dangerous city in the...,Black,-0.205019,296,"['memphis', '26th', 'dangerous', 'city', 'enti...",0.112700,73.173333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,t5_2rfyw,forfactorial,compare the message from wandering earth 2 and...,compare the message from wandering earth 2 and...,Asian,-0.522514,89,"['compare', 'message', 'wandering', 'earth', '...",0.070321,11.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,t5_2rfyw,sojuandbbq,If you’re serious or semi-serious you could lo...,If youre serious or semiserious you could look...,Asian,0.968534,562,"['if', 'youre', 'serious', 'semiserious', 'cou...",0.059412,116.640000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369489,t5_2qr1c,ApacheNDN,Thanks. I'll tell her.,Thanks Ill tell her,Native American,-0.417131,22,"['thanks', 'ill', 'tell']",0.021955,4.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
369490,t5_2qr1c,EternityOnDemand,I heard a black guy here in Canada telling me ...,I heard a black guy here in Canada telling me ...,Native American,0.016193,219,"['i', 'heard', 'black', 'guy', 'canada', 'tell...",0.042735,224.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
369491,t5_2r81q,Mace-Window_777,Did you report it to YouTube administration? T...,Did you report it to YouTube administration Th...,Black,-0.517575,78,"['did', 'report', 'youtube', 'administration',...",-0.097409,13.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
369492,t5_2rfz1,smell-the-roses,Zebra people?,Zebra people,White,-0.545455,13,"['zebra', 'people']",-0.009624,2.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(data['pos_tags'])

In [ ]:
tag_count_data

In [ ]:
new_data = pd.concat([data, tag_count_data], axis=1).fillna(0).drop(['pos_tags'], axis=1)

In [ ]:
# Fetching unique word count


def ngram_unique(text):

    unique = []
    for word in text:
        if word not in unique:
            unique.append(word)
    
    return len(unique)/len(text)

In [ ]:
data['unique_words'] = data['token_text'].progress_apply(ngram_unique)

In [ ]:
print(data['unique_words'])

In [ ]:
print(new_data.columns)
new_data.to_csv("/DATA/Processed/racial_groups.csv", index=False)